In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.f_sto_pending_21d 
 TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) AS
WITH 
time_range AS (
    SELECT EXPLODE(SEQUENCE(
        DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 21),
        DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), 1),
        INTERVAL 1 DAY
    )) AS calday
),
pick_pack_spr AS (
    SELECT 
        a.wh_site_id AS dc_id,
        sku,
        s.external_code AS sto_number,
        SUM(a.base_qty) AS qty
    FROM {catalog_name}.udp_wcm_silver_supra.f_in_out_transaction_spr a
    INNER JOIN time_range tr ON tr.calday = DATE(a.tran_date)
    LEFT JOIN {catalog_name}.udp_wcm_silver_supra.d_so_header_spr s ON s.so_code = a.object_code
    WHERE a.object_code LIKE 'SO%' 
      AND a.object_type = 'SO' 
      AND a.job_type = 'SO_AUTOPP_PICKING'
    GROUP BY 1,2,3
),
spr_dc AS (
    SELECT DISTINCT dc_id FROM pick_pack_spr
),
dc_site AS (
    SELECT 
        st.*, 
        CASE WHEN s.dc_id IS NOT NULL THEN 1 ELSE 0 END AS spr_dc
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st
    LEFT JOIN spr_dc s ON s.dc_id = st.store_id
    WHERE st.business_unit = '1100' 
      AND st.store_status_id = 'OPEN'
),
po_allocation AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmpo08
    QUALIFY ROW_NUMBER() OVER (PARTITION BY ebeln, ebelp ORDER BY proc_date DESC) =1      
  
),
flow_through AS (
    SELECT ebeln AS po_doc, ebelp AS po_item
    FROM po_allocation
    WHERE abeln IS NOT NULL AND aurel = 3
),
sto AS (
    SELECT 
        a.po_doc AS sto_number,
        a.po_item AS sto_item,
        a.product_id,
        a.dc_id,
        c.spr_dc,
        a.creation_date AS calday,
        SUM(a.po_qty) AS po_qty
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
    INNER JOIN dc_site c ON a.dc_id = c.store_id
    INNER JOIN time_range tr ON tr.calday = a.creation_date
    LEFT JOIN flow_through ft ON a.po_doc = ft.po_doc AND a.po_item = ft.po_item
    WHERE a.po_type = 'Stock Transp. Order' 
      AND a.compl_del IS NULL 
      AND ft.po_doc IS NULL
    GROUP BY 1,2,3,4,5,6
),
outbound AS (
    SELECT  
        plant AS dc_id,
        oi_ebeln AS sto_number,
        oi_ebelp AS sto_item,
        material AS product_id,
        SUM(CASE WHEN dcindic = 'H' THEN 1 ELSE -1 END * ABS(cpquabu)) AS gi_qty
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmmim03 m
    INNER JOIN dc_site st ON st.store_id = m.plant
    INNER JOIN time_range tr ON tr.calday = m.calday
    WHERE movetype IN ('641','642','351','352') 
      AND stocktype = 'A'
      AND m.segment IN ('1100', '1500', '2000')
    GROUP BY 1,2,3,4
),
pick_pack_sap AS (
    SELECT 
        p.dc_id,
        p.product_id,
        p.sto_number,
        SUM(p.deliver_qty_base) AS qty
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_delivery_order p
    INNER JOIN dc_site st ON st.store_id = p.dc_id
    INNER JOIN time_range tr ON tr.calday = p.calday
    WHERE st.spr_dc = 0 AND p.sts_pick = 'C'
    GROUP BY 1,2,3
),
pick_pack AS (
    SELECT 
        dc_id, product_id, sto_number, SUM(qty) AS qty
    FROM (
        SELECT dc_id, product_id, qty, sto_number FROM pick_pack_sap
        UNION ALL
        SELECT dc_id, sku AS product_id, qty, sto_number FROM pick_pack_spr
    ) all_pp
    GROUP BY 1,2,3
)

SELECT 
    sto.calday,
    sto.dc_id,
    sto.spr_dc,
    sto.sto_number,
    sto.sto_item,
    sto.product_id,
    sto.po_qty,
    IFNULL(outbound.gi_qty, 0) AS gi_qty,
    IFNULL(pp.qty, 0) AS qty_picked
FROM sto
LEFT JOIN outbound 
    ON sto.sto_number = outbound.sto_number
   AND sto.sto_item = outbound.sto_item
   AND sto.product_id = outbound.product_id
LEFT JOIN pick_pack pp 
    ON pp.sto_number = sto.sto_number
   AND pp.dc_id = sto.dc_id
   AND pp.product_id = sto.product_id
WHERE IFNULL(outbound.gi_qty, 0) = 0""")